In [9]:
import numpy as np
import pandas as pd
from typing import List


In [249]:
# Super basic function to create an 'Exercise' line

def exerciseLine(name: str, sets: int, reps: int, weight) -> str:
    return f'{name}: {sets}x{reps}, {weight}lbs'

def exerciseLineFromReps(name: str, reps: List[int], weight) -> str:
    if len(set(reps)) == 1:
        return f'{name}: {len(reps)}x{reps}, {weight}lbs'
    else:
        repStr = '/'.join([str(rep) for rep in reps])
        return f'{name}: {repStr}, {weight}lbs'

def generateHeader(name: str, program: str, date: str):
    # m/dd: AT 'PROGRAM' week 'w' day 'd'
    # How do we get week and day from strong data...?
    return f"{date}: {name} {program} week X day X"

# For List[Weight] and List[List[Reps]], generate a line
# s["Reps list"] = [[a, b], [c, d]]
# s["Weight"] = [x, y]
def generateLineFromSeries(s):
    name, repsList, weights = s["Exercise Name"], s["Reps"], s["Weight"]
    if name == "Intensity":
        return f"Intensity: {repsList[0][0]}"
    
    blocks = []
    for weight, reps in zip(weights, repsList):
        weight = str(weight).replace('.0', '')
        block = ''
        # Amrap rule: 4x4 -> 8 instead of 4/4/4/4/8
        if len(set(reps)) == 2 and len(set(reps[:-1])) == 1:
            block = f'{len(reps)-1}x{reps[0]} -> {reps[-1]}, {weight}lbs'
        
        # When we have 1 rep count for all sets
        elif len(set(reps)) == 1:
            block = f'{len(reps)}x{reps[0]}, {weight}lbs'
        else:
            repStr = '/'.join([str(rep) for rep in reps])
            block = f'{repStr}, {weight}lbs'

        blocks.append(block)
    
    meat = " | ".join(blocks)
    return f'{name}: {meat}'

In [250]:
df = pd.read_csv("strong.csv", na_filter=False)
df["Date"] = pd.to_datetime(df["Date"])

# To cut short
df = df.tail(10)
# Group by Date, Exercise and Weight. Can agg reps using .apply.
df = df.groupby(["Date", "Exercise Name", "Weight"])["Reps"].agg(list).reset_index()
df = df.groupby(["Date", "Exercise Name"]).agg(list).reset_index()

# Generate caption lines!
df["Exercise line"] = df.apply(generateLineFromSeries, axis=1)

# df.group

# Real ugly but converts date_time to our format m/d. Do this AFTER so dates on different years don't get clumped
# df["Date parsed"] = df["Date"].dt.strftime("-%m/-%d").str.replace("-0", "-").str.replace("-", "")
df

,Date,Exercise Name,Weight,Reps,Exercise line,Date parsed
0,2022-12-06 08:57:32,Bench Press - Close Grip (Barbell),[205.0],"[[6, 6]]","Bench Press - Close Grip (Barbell): 2x6, 205lbs",12/6
1,2022-12-06 08:57:32,Bicep Curl (Cable),"[65.0, 65.5]","[[10, 10], [10]]","Bicep Curl (Cable): 2x10, 65lbs | 1x10, 65.5lbs",12/6
2,2022-12-06 08:57:32,Deadlift (Barbell),[330.0],"[[4, 4, 4, 8]]","Deadlift (Barbell): 3x4 -> 8, 330lbs",12/6
3,2022-12-06 08:57:32,Intensity,[0.0],[[7]],Intensity: 7,12/6


In [180]:
reps = [4, 4, 4, 4, 8]

dog


'abcdef'